In [13]:
# from CBAStats.Player import *
# from CBAStats.Team import *
# from CBAStats.Player import stats_output
from pathlib import Path
from sqlalchemy import create_engine
import pymysql
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
import pandas as pd
import datetime
import numpy as np
import re
import time

In [14]:
def get_page_content(url, encoding='UTF-8', header={
            'User-Agent': r'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                          r'Chrome/41.0.2227.1 Safari/537.36'}):
    session = requests.Session()
    base_url = url
    response = session.get(base_url, headers=header)
    response.encoding = encoding
    page_content = BeautifulSoup(response.content, "html.parser")
    return page_content

# 爬取参数列表

In [15]:
# 链接中有几个参数，qleagueid是赛季，qmonth是月，qteamid是球队
# 空出qmonth和qteamid则可以无差别选取某赛季的所有比赛
# qleagueid并不是逐一递增或递减的，如20192020赛季是205，20182019是198

def get_params(default_schedule_url = "http://cba.sports.sina.com.cn/cba/schedule/all/"):
    """
    从赛程页爬取赛季，月，球队的可能参数值。
    从可能的参数值里选取想要爬取的赛季，月，球队等，并用get_url函数拼凑出目标url。
    """
    param_html_list = get_page_content(url=default_schedule_url).find_all('select')
    param_dict = {}

    for param in param_html_list:
        options = {}
        for option in param.find_all('option'):
            options[option.text] = option['value']
        param_dict[param['name']] = options
    
    return param_dict

# 爬取赛程、url

In [16]:
def get_url(leagueid = '19-20',month = '全部',teamid ='全部'):

    """
    此函数用于拼凑想要爬取的目标url。
    
    链接中有几个参数，qleagueid是赛季，qmonth是月，qteamid是球队。
    qleagueid并不是逐一递增或递减的，如20192020赛季是205，20182019是198。
    
    """
    param_dict = get_params(default_schedule_url = "http://cba.sports.sina.com.cn/cba/schedule/all/")
    qleagueid=param_dict['qleagueid'][leagueid]
    qmonth=param_dict['qmonth'][month]
    qteamid=param_dict['qteamid'][teamid]
    scrape_url = f"http://cba.sports.sina.com.cn/cba/schedule/all/?qleagueid={qleagueid}&qmonth={qmonth}&qteamid={qteamid}"
    
    return scrape_url

In [34]:
def scrape_schedule(season = '19-20',month = '全部',teamid ='全部', 
                    only_show_params = False,param_url = "http://cba.sports.sina.com.cn/cba/schedule/all/"):
    """
    此函数用于爬取赛程和详细数据的链接，并存入CBA_Data.Staging_Schedules
    
    请注意，season(网页使用的参数是qleagueid)，month，team参数值是有限定值的。
    可通过运行scrape_schedule(only_show_params = False)来查询可用的参数。如果only_show_params=True那么本函数不会爬取赛程数据，只会显示可用参数。
    
    参数中，season(qleagueid)是赛季，qmonth是月，qteamid是球队。
    season(qleagueid)，如20192020赛季是205，20182019是198。
    
    Parameters: 
    
    season:
    month:
    teamid:
    only_show_params:
    param_url:
    """
    
    param_dict = get_params(default_schedule_url = param_url)
    
    if only_show_params:
        return param_dict
    
    # 拼凑出目标url
    schedule_url = get_url(leagueid = season,month = month,teamid =teamid)
    
    # 爬取整张页面html
    page_content = get_page_content(url=schedule_url)

    # 赛程页面共有两张表
    # 第一张表是当前轮次比赛
    # 第二张表才是该赛季所有比赛

    # 爬取整张表的html
    target_table = page_content.find_all("table")[1]

    # 获取表头
    headers = [th.text for th in target_table.find('thead').find_all('th')]

    # 获取表格数据的html
    tbody = target_table.find('tbody')

    # 获取表格每行的html，存入list
    trs = tbody.find_all('tr')
    
    # 用于存储文本的list
    text_list =[]
    # 用于存储链接的list
    link_list = []

    for tr in trs:
        # 从每行中获取每一单元格的html
        tds = tr.find_all('td')
        for td in tds:
            # 获取每单元格的纯文本内容
            cell_text = str(td.text).strip()
            # 单元格内若无链接则为空字符
            cell_link = ''
            if td.find('a',href=True):
                # 单元格内存在链接则保存
                cell_link = td.find('a',href=True)['href'].strip()
            text_list.append(cell_text)
            link_list.append(cell_link)
    
    # 分别将文本和链接保存在两个dataframe中，最后再横向合并
    # 链接的dataframe在column header后加上“_link“后缀
    # 因此会有空白列，此处不删除是考虑到未来可能会有新内容
    text_list = np.reshape(text_list, [-1, 10])
    link_list = np.reshape(link_list, [-1, 10])
    df_schedule_text = pd.DataFrame(data=text_list, columns=headers)
    df_schedule_link = pd.DataFrame(data=link_list, columns=[header + '_link' for header in headers])

    df_schedule_full = pd.merge(df_schedule_text, df_schedule_link, left_index=True, right_index=True)

    df_schedule_full['SinaGame_ID'] = df_schedule_full['统计_link'].apply(lambda x: int(re.findall('show[/](\d+)[/]', x)[0]))
    df_schedule_full['客队ID'] = df_schedule_full['客队'].apply(lambda x: int(param_dict['qteamid'][x]))
    df_schedule_full['主队ID'] = df_schedule_full['主队'].apply(lambda x: int(param_dict['qteamid'][x]))
    df_schedule_full['日期'] = pd.to_datetime(df_schedule_full['日期'])
       
    return df_schedule_full

# 爬取详细数据

1. 爬取完整新schedule
1. 爬取数据库内现存的schedule
2. 在新schedule中删除未结束的比赛
    * 比分仍是‘VS’ 或者
    * 现在日期-比赛日期<3小时 （防止比赛正在进行中）
3. 找出新比赛
    * 对比新schedule和现存schedule的SinaGame_ID，找出新的ID （防止新浪采用新的GameID） 且
    * 没有相同对手在相同日期的，已结束的比赛
4. 开始爬取新比赛的details
1. 每爬完一个比赛，写入上次爬取时间，再写入数据库内现存的schedule

In [36]:
from sqlalchemy import create_engine
import pymysql

def get_existing_schedule():
    
    user_name = 'master'
    passcode = 'Pw#cbashuju0131'
    endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
    database = 'CBA_Data'
    engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
    connection= engine.connect()

    # df.to_sql(name='Schedules',con=connection,index=False,if_exists='replace')
    df = pd.read_sql("select * from CBA_Data.Schedules", connection)
    connection.close()
    return df

def get_staging_schedule():
    
    user_name = 'master'
    passcode = 'Pw#cbashuju0131'
    endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
    database = 'CBA_Staging'
    engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
    connection= engine.connect()

    # df.to_sql(name='Schedules',con=connection,index=False,if_exists='replace')
    df = pd.read_sql("select * from CBA_Staging.Schedules", connection)
    connection.close()
    return df

def load_schedule_into_staging(scraped_schedule):
    # 写入数据库Staging_Schedule
    user_name = 'master'
    passcode = 'Pw#cbashuju0131'
    endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
#     database = 'CBA_Data'
    database = 'CBA_Staging'
    engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
    connection= engine.connect()

    scraped_schedule.to_sql(name='Schedules',con=connection,index=False,if_exists='replace')
    connection.close()

def clean_staging_schedule():
    
    # query to clean-up staging schedule
    
    user_name = 'master'
    passcode = 'Pw#cbashuju0131'
    endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
    database = 'CBA_Staging'
    engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
    with engine.connect() as connection:
        with connection.begin():
            # delete未开始，没比分的比赛
            connection.execute("""
            DELETE FROM CBA_Staging.Schedules
            WHERE 比分='VS';
                               """)
            
            # delete有相同ID的比赛
            connection.execute("""
            DELETE
            FROM CBA_Staging.Schedules
            WHERE CBA_Staging.Schedules.SinaGame_ID IN (
            SELECT SinaGame_ID
            FROM CBA_Data.Schedules
            );

                               """)
            
            
            # delete同一天同样队伍有的比赛
            connection.execute("""
            DELETE
            FROM CBA_Staging.Schedules
            WHERE EXISTS (
            SELECT 1
            FROM  CBA_Data.Schedules prod
            WHERE prod.主队=CBA_Staging.Schedules.主队
            AND prod.主队=CBA_Staging.Schedules.主队
            AND prod.日期=CBA_Staging.Schedules.日期
            );
                               """)
            




### 获取需要scrape的schedule

In [77]:
scraped_schedule = scrape_schedule()
load_schedule_into_staging(scraped_schedule)

clean_staging_schedule()

staging_schedule = get_staging_schedule()
staging_schedule

,轮次,日期,主队,比分,客队,战报,统计,组图,地点,电视,...,比分_link,客队_link,战报_link,统计_link,组图_link,地点_link,电视_link,SinaGame_ID,客队ID,主队ID
0,半决赛,2020-08-04 20:00:00,广东,111:109,北京,战报,统计,组图,青岛,,...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/10/,https://sports.sina.com.cn/basketball/cba/2020...,http://cba.sports.sina.com.cn/cba/schedule/sho...,[CBA半决赛]广东111-109北京,,,19012,10,1


### 开始爬取 （先把existing schedule给scrape了，再按正常流程scrape staging schedule里的比赛）

In [28]:
# 共同datapoint
round_name = row['轮次']
game_UID=row['Game_ID']
hteam_ID = row['主队ID']
ateam_ID = row['客队ID']
game_hteam_name = row['主队']
game_ateam_name = row['客队']
location = row['地点']

base_url = row['统计_link']

page_content = get_page_content(base_url, encoding='GB2312')
page_content

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>广东VS辽宁_CBA联赛单场比分_中国篮球数据库_篮球-CBA_新浪体育_新浪网</title>
<meta content="广东VS辽宁CBA联赛赛程赛果，CBA联赛直播，CBA联赛赛程赛果，CBA联赛直播，中国篮球数据库" name="keywords"/>
<meta content="新浪CBA联赛，提供CBA联赛最新赛程赛果、CBA联赛直播、CBA联赛总决赛、CBA联赛录像、CBA联赛视频集锦、CBA联赛球队球员、CBA联赛十佳球、CBA联赛快讯、CBA联赛图片等丰富的CBA联赛新闻和数据。" name="description"/>
<link href="http://www.sinaimg.cn/ty/deco/2013/0124/cbacss/css/base.css" rel="stylesheet" type="text/css"/>
<link href="http://n.sinaimg.cn/default/7919a7ba/20190305/score.css" rel="stylesheet" type="text/css"/>
<script src="http://www.sinaimg.cn/ty/deco/2013/0124/cbacss/css/cba.js" type="text/javascript"></script>
</head>
<body>
<!-- wrap begin -->
<div class="wrap" id="wraper">
<style type="text/css">
	.sHBorder {
    border: 1px solid #E3E3E3;
    overflow: hidden;


In [25]:
def get_gamestats():
    user_name = 'guest'
    passcode = 'Guest123456'
    endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
    database = 'CBA_Data'
    engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')

    connection= engine.connect()
    df = pd.read_sql("select * from CBA_Data.PlayerStatsPerGame", connection)
    connection.close()
    return df